# 第2.5回講義 演習

In [ ]:
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.datasets import mnist


import numpy as np

np.random.seed(34)

## 目次

課題 1. 多層パーセプトロンの実装と学習 (XOR)
1. 活性化関数とその微分
2. データセットの設定と重みの定義
3. train関数とvalid関数
4. 学習

課題 2. 多層パーセプトロンの実装と学習 (MNIST)
1. ソフトマックス関数
2. データセットの設定
3. 全結合層の定義
4. train関数とvalid関数
5. 学習

課題 3. 数値微分 (勾配チェック)
1. 1変数の場合
2. 多変数の場合 (MLP)

## 課題 1. 多層パーセプトロンの実装と学習 (XOR)

### 1. 活性化関数とその微分

#### 1.1. sigmoid

\begin{align}
    \sigma (x) &= \frac{1}{1 + \exp(-x)} \\
    \sigma' (x) &= \sigma(x) (1 - \sigma(x))
\end{align}

In [ ]:
def sigmoid(x):
    return np.tanh(x * 0.5) * 0.5 + 0.5

def deriv_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

#### 1.2. ReLU

\begin{align}
    \mathrm{ReLU}(x) = \max (0, x)
\end{align}

\begin{align}
    \mathrm{ReLU}'(x) =
    \begin{cases}
    1 \hspace{10mm} \text{if} \hspace{5mm} x > 0 \\
    0 \hspace{10mm} \text{otherwise}
    \end{cases}
\end{align}

In [ ]:
def relu(x):
    return np.maximum(x, 0)

def deriv_relu(x):
    return (x > 0).astype(x.dtype)

#### 1.3. tanh

\begin{align}
    \tanh(x) = \frac{\exp(2x) - 1}{\exp(2x) + 1}
\end{align}

\begin{align}
    \tanh'(x) = 1 - \tanh^2(x)
\end{align}

In [ ]:
def tanh(x):
    return np.tanh(x)

def deriv_tanh(x):
    return 1 - np.tanh(x)**2

### 2. データセットの設定と重みの定義

In [ ]:
# XORデータセット
x_train_xor = np.array([[0, 1], [1, 0], [0, 0], [1, 1]])
y_train_xor = np.array([[1], [1], [0], [0]])
x_valid_xor, y_valid_xor = x_train_xor, y_train_xor

# 重み (入力層の次元数: 2, 隠れ層の次元数: 8, 出力層の次元数: 1)
W1 = np.random.uniform(low=-0.08, high=0.08, size=(2, 8)).astype('float32')
b1 = np.zeros(8).astype('float32')
W2 = np.random.uniform(low=-0.08, high=0.08, size=(8, 1)).astype('float32')
b2 = np.zeros(1).astype('float32')

### 3. train関数とvalid関数
隠れ層と出力層の2層からなるMLPを実装していきます。

数式は以下のようになります。

#### 誤差関数
負の対数尤度 (交差エントロピー)

\begin{align}
    E({\bf \hat{y}}, {\bf y}) = -\frac{1}{N}\sum^N_{i=1} \left[{\bf y}_i \log {\bf \hat{y}}_i + (1 - {\bf y}_i) \log(1 - {\bf \hat{y}}_i)\right]
\end{align}

#### 順伝播
\begin{align*}
    {\bf u}^{(1)}_i &= {\bf W}^{(1)\mathrm{T}}{\bf x}_i + {\bf b}^{(1)} &\text{(隠れ層)} \\
    {\bf h}^{(1)}_i &= \mathrm{ReLU}({\bf u}^{(1)}_i) &\text{(隠れ層)} \\
    {\bf u}^{(2)}_i &= {\bf W}^{(2)\mathrm{T}}{\bf h}^{(1)}_i + {\bf b}^{(2)} &\text{(出力層)} \\
    {\bf \hat{y}}_i &= \sigma({\bf u}^{(2)}_i) &\text{(出力層)}
\end{align*}

#### 逆伝播
\begin{align*}
    {\bf \delta}^{(2)}_i &= {\bf \hat{y}}_i - {\bf y}_i &\text{(出力層)} \\
    {\bf \delta}^{(1)}_i &= \mathrm{ReLU}'({\bf u}^{(1)}_i) \odot ({\bf W}^{(2)\mathrm{T}}\delta^{(2)}_i) &\text{(隠れ層)}
\end{align*}

#### 勾配の計算
\begin{align*}
    \nabla_{{\bf W}^{(1)}}E &= \frac{1}{N}\sum^N_{i=1}\delta^{(1)}_i {\bf x}^{\mathrm{T}}_i &\text{(隠れ層)} \\
    \nabla_{{\bf b}^{(1)}}E &= \frac{1}{N}\sum^N_{i=1}\delta^{(1)}_i &\text{(隠れ層)} \\
    \nabla_{{\bf W}^{(2)}}E &= \frac{1}{N}\sum^N_{i=1}\delta^{(2)}_i {\bf h}^{(1)\mathrm{T}}_i &\text{(出力層)} \\
    \nabla_{{\bf b}^{(2)}}E &= \frac{1}{N}\sum^N_{i=1}\delta^{(2)}_i &\text{(出力層)}
\end{align*}

#### 重みの更新
\begin{align*}
    {\bf W}^{(1)} &\leftarrow {\bf W}^{(1)} - \epsilon \nabla_{{\bf W}^{(1)}} E &\text{(隠れ層)} \\
    {\bf b}^{(1)} &\leftarrow {\bf b}^{(1)} - \epsilon \nabla_{{\bf b}^{(1)}} E &\text{(隠れ層)} \\
    {\bf W}^{(2)} &\leftarrow {\bf W}^{(2)} - \epsilon \nabla_{{\bf W}^{(2)}} E &\text{(出力層)} \\
    {\bf b}^{(2)} &\leftarrow {\bf b}^{(2)} - \epsilon \nabla_{{\bf b}^{(2)}} E &\text{(出力層)}
\end{align*}

In [ ]:
# logの中身が 0になるのを防ぐ
def np_log(x):
    return np.log(np.clip(x, 1e-10, 1e+10))

In [ ]:
def train_xor(x, y, eps):
    """
    :param x: np.ndarray, 入力データ, shape=(batch_size, 入力層の次元数)
    :param t: np.ndarray, 教師ラベル, shape=(batch_size, 出力層の次元数)
    :param eps: float, 学習率
    """
    global W1, b1, W2, b2
    
    batch_size = x.shape[0]
    
    # 順伝播
    u1 = np.matmul(x, W1) + b1 # shape: (batch_size, 隠れ層の次元数)
    h1 = relu(u1)

    u2 = np.matmul(h1, W2) + b2 # shape: (batch_size, 出力層の次元数)
    y_hat = sigmoid(u2)
    
    # 誤差の計算
    cost = (- y * np_log(y_hat) - (1 - y) * np_log(1 - y_hat)).mean()
    
    # 逆伝播
    delta_2 = y_hat - y # shape: (batch_size, 出力層の次元数)
    delta_1 = deriv_relu(u1) * np.matmul(delta_2, W2.T) # shape: (batch_size, 隠れ層の次元数)

    # 勾配の計算
    dW1 = np.matmul(x.T, delta_1) / batch_size # shape: (入力層の次元数, 隠れ層の次元数)
    db1 = np.matmul(np.ones(batch_size), delta_1) / batch_size # shape: (隠れ層の次元数,)
    
    dW2 = np.matmul(h1.T, delta_2) / batch_size # shape: (隠れ層の次元数, 出力層の次元数)
    db2 = np.matmul(np.ones(batch_size), delta_2) / batch_size # shape: (出力層の次元数,)

    # パラメータの更新
    W1 -= eps * dW1
    b1 -= eps * db1
    
    W2 -= eps * dW2
    b2 -= eps * db2

    return cost

def valid_xor(x, y):
    global W1, b1, W2, b2
    
    # 順伝播
    u1 = np.matmul(x, W1) + b1
    h1 = relu(u1)
    
    # 逆伝播
    u2 = np.matmul(h1, W2) + b2
    y_hat = sigmoid(u2)
    
    # 誤差の計算
    cost = (- y * np_log(y_hat) - (1 - y) * np_log(1 - y_hat)).mean()
    
    return cost, y_hat

### 4. 学習

In [ ]:
for epoch in range(3000):
    x_train_xor, y_train_xor = shuffle(x_train_xor, y_train_xor)
    cost = train_xor(x_train_xor, y_train_xor, 0.05)
    cost, y_pred = valid_xor(x_valid_xor, y_valid_xor)

print(y_pred)

## 課題 2. 多層パーセプトロンの実装と学習 (MNIST)

### 1. ソフトマックス関数

\begin{align*}
    \mathrm{softmax}({\bf x})_k = \frac{\exp({\bf x}_k)}{\sum^K_{k'=1}\exp({\bf x}_{k'})} \hspace{10mm} \text{for} \hspace{3mm} k = 1, \ldots, K
\end{align*}

\begin{align*}
    \mathrm{softmax}({\bf x})_k = \mathrm{softmax}({\bf x})_k (1 - \mathrm{softmax}({\bf x}))_k
\end{align*}

In [ ]:
def softmax(x):
    x -= x.max(axis=1, keepdims=True)
    x_exp = np.exp(x)
    return x_exp / np.sum(x_exp, axis=1, keepdims=True)

def deriv_softmax(x):
    return softmax() * (1 - softmax(x))

### 2. データセットの設定

In [ ]:
(x_mnist_1, y_mnist_1), (x_mnist_2, y_mnist_2) = mnist.load_data()

x_mnist = np.r_[x_mnist_1, x_mnist_2]
y_mnist = np.r_[y_mnist_1, y_mnist_2]

x_mnist = x_mnist.astype('float32') / 255.
y_mnist = np.eye(N=10)[y_mnist.astype('int32').flatten()]

x_mnist=x_mnist.reshape(x_mnist.shape[0],-1)

x_train_mnist, x_test_mnist, y_train_mnist, y_test_mnist = train_test_split(x_mnist, y_mnist, test_size=10000)
x_train_mnist, x_valid_mnist, y_train_mnist, y_valid_mnist = train_test_split(x_train_mnist, y_train_mnist, test_size=10000)


### 3. 全結合層の定義

多層のMLPを実装していけるよう、全結合層をクラスとして定義していきます。

順伝播、逆伝播、勾配の計算をそれぞれ関数として実装します。

数式は以下のようになります。

#### 順伝播 (```__call__```)
\begin{align*}
    {\bf u}^{(l)}_i &= {\bf W}^{(l)\mathrm{T}} {\bf h}^{(l-1)}_i + {\bf b}^{(l)} \\
    {\bf h}^{(l)}_i &= \mathrm{function}({\bf u}^{(l)}_i)
\end{align*}

#### 逆伝播 (```b_prop```)
\begin{align*}
    \delta^{(l)}_i = \mathrm{function}'({\bf u}^{(l)}_i) \odot ({\bf W}^{(l+1)\mathrm{T}} \delta^{(l+1)}_i)
\end{align*}

#### 勾配の計算 (```compute_grad```)
\begin{align*}
    \nabla_{{\bf W}^{(l)}}E &= \frac{1}{N}\sum^N_{i=1}\delta^{(l)}_i {\bf h}^{(l)\mathrm{T}}_i \\
    \nabla_{{\bf b}^{(l)}}E &= \frac{1}{N}\sum^N_{i=1}\delta^{(l)}_i \\
\end{align*}

`get_params`、`set_params`、`get_grads`はそれぞれ重み、勾配をベクトルで受け渡す関数です。
課題3の勾配チェックの際に使用します。

In [ ]:
class Dense:
    def __init__(self, in_dim, out_dim, function, deriv_function):
        self.W = np.random.uniform(low=-0.08, high=0.08,
                                   size=(in_dim, out_dim)).astype('float64')
        self.b = np.zeros(out_dim).astype('float64')
        self.function = function
        self.deriv_function = deriv_function
        
        self.x = None
        self.u = None
        
        self.dW = None
        self.db = None

        self.params_idxs = np.cumsum([self.W.size, self.b.size])

    def __call__(self, x):
        self.x = x
        self.u = np.matmul(self.x, self.W) + self.b
        return self.function(self.u)

    def b_prop(self, delta, W):
        self.delta = self.deriv_function(self.u) * np.matmul(delta, W.T)
        return self.delta
    
    def compute_grad(self):
        batch_size = self.delta.shape[0]
        
        self.dW = np.matmul(self.x.T, self.delta) / batch_size
        self.db = np.matmul(np.ones(batch_size), self.delta) / batch_size

    def get_params(self):
        return np.concatenate([self.W.ravel(), self.b], axis=0)
    
    def set_params(self, params):
        _W, _b = np.split(params, self.params_idxs)[:-1]
        self.W = _W.reshape(self.W.shape)
        self.b = _b
    
    def get_grads(self):
        return np.concatenate([self.dW.ravel(), self.db], axis=0)

MLP全体の順伝播・逆伝播を行う関数をそれぞれ実装します。

In [ ]:
def f_props(layers, x):
    for layer in layers:
        x = layer(x)
    return x

In [ ]:
def b_props(layers, delta):
    batch_size = delta.shape[0]
    
    for i, layer in enumerate(layers[::-1]):
        if i == 0: # 出力層の場合
            layer.delta = delta # y - t
            layer.compute_grad() # 勾配の計算
        else: # 出力層以外の場合
            delta = layer.b_prop(delta, W) # 逆伝播
            layer.compute_grad() # 勾配の計算

        W = layer.W

パラメータの更新を行う関数を実装します。

In [ ]:
def update_params(layers, eps):
    for layer in layers:
        layer.W -= eps * layer.dW
        layer.b -= eps * layer.db

最後にMLPを定義します。

In [ ]:
layers = [
    Dense(784, 100, relu, deriv_relu),
    Dense(100, 100, relu, deriv_relu),
    Dense(100, 10, softmax, deriv_softmax)
]

### 4. train関数とvalid関数

#### 誤差関数

負の対数尤度 (多クラス交差エントロピー)

\begin{align*}
    E({\bf \hat{y}}, {\bf y}) = -\frac{1}{N}\sum^N_{i=1}\sum^K_{k=1} {\bf y}_{i, k}\log{\bf \hat{y}}_{i, k}
\end{align*}

In [ ]:
def train_mst(x, y, eps=0.01):
    # 順伝播
    y_hat = f_props(layers, x)

    # 誤差の計算
    cost = (- y * np_log(y_hat)).sum(axis=1).mean()
    
    # 逆伝播
    delta = y_hat - y
    b_props(layers, delta)

    # パラメータの更新
    update_params(layers, eps)

    return cost

In [ ]:
def valid_mst(x, y):
    # 順伝播
    y_hat = f_props(layers, x)
    
    # 誤差の計算
    cost = (- y * np_log(y_hat)).sum(axis=1).mean()
    
    return cost, y_hat

### 5. 学習

In [ ]:
for epoch in range(3):
    x_train_mnist, y_train_mnist = shuffle(x_train_mnist, y_train_mnist)
    # オンライン学習
    for x, y in zip(x_train_mnist, y_train_mnist):
        cost = train_mst(x[None, :], y[None, :], eps=0.01)
    
    cost, y_pred = valid_mst(x_valid_mnist, y_valid_mnist)
    accuracy = accuracy_score(y_valid_mnist.argmax(axis=1), y_pred.argmax(axis=1))
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(epoch + 1, cost, accuracy))

## 課題 3. 数値微分 (勾配チェック)

誤差逆伝播法による勾配の計算は少し複雑なため、実装にバグが入りがちです。

実装が簡単な数値微分と結果を比較することで、逆伝播の実装が正しいかを確認してみましょう。

### 1. 1変数の場合

まず肩慣らしに簡単な2次関数に対して数値微分を行ってみましょう。

In [ ]:
def f(x):
    return x**2

def deriv_f(x):
    return 2 * x

1変数の場合の数値微分の式は次のようになります。

$$
    f'(x) = \lim_{h \rightarrow 0} \frac{f(x + h) - f(x - h)}{2h}
$$

In [ ]:
eps = 1e-5
x = 2.0

grad_auto = deriv_f(x)
grad_num = (f(x + eps) - f(x - eps)) / (2 * eps)

grad_auto, grad_num

### 2. 多変数の場合 (MLP)

次に課題2で定義したMLPに対して数値微分の計算を行い、誤差逆伝播による勾配 (`dW`、`db`) の計算が間違っていないかを確認してみましょう。

多変数 (MLP) の場合の数値微分の式は次のようになります。ある1つの変数$\theta_m$のみを$h$だけ動かした場合を考えます。

$$
    \frac{\partial E}{\partial \theta_m} = \lim_{h\rightarrow 0} \frac{E(\theta_1, \theta_2, \ldots, \theta_m + h, \ldots, \theta_M) - E(\theta_1, \theta_2, \ldots, \theta_m - h, \ldots, \theta_M)}{2h}
$$

実装では、変数全体のサイズのゼロベクトルを用意し、$m$番目の要素のみ$h$だけずらされたベクトルを作りそれに対応する誤差を計算し、そこから上の式に従って最終的な微分の値を求めていきます。

まず各層ごとの重みをベクトルで取得しリストで保存していく関数を実装していきます。

MLPの各レイヤーから重みをベクトルで取得するために、先程`Dense`クラスで定義した`set_params`、`get_params`を使用します。

In [ ]:
def get_params(layers):
    params_all = []
    for layer in layers:
        params = layer.get_params()
        params_all.append(params)
    
    return params_all

In [ ]:
def set_params(layers, params_all):
    for layer, params in zip(layers, params_all):
        layer.set_params(params)

In [ ]:
def compute_cost(x, y):
    # 順伝播
    y_hat = f_props(layers, x)
    
    # 誤差の計算
    cost = (- y * np_log(y_hat)).sum(axis=1).mean()
    
    return cost

勾配の計算に使用するデータを用意します。勾配チェックの際は`float32`ではなく`float64`を使いましょう。

In [ ]:
batch_size = 32

x = x_train_mnist[:batch_size].astype('float64')
y = y_train_mnist[:batch_size].astype('float64')

In [ ]:
eps = 1e-5

params_all = get_params(layers)
grads_all_num = []

# レイヤーごとに勾配を計算
for layer, params in zip(layers, params_all):
    shift = np.zeros_like(params)
    grads_num = np.zeros_like(params)
    
    # レイヤー内のM個のパラメータに対してそれぞれ微分を計算する
    for m in range(len(params)):
        shift[m] = eps # m番目のパラメータのみeps分ずらす [0, 0, ..., 0, eps, 0, ..., 0]
        
        params_right = params + shift
        layer.set_params(params_right)
        cost_right = compute_cost(x, y) # L(x; ..., \theta_m + eps, ...)
        
        params_left = params - shift
        layer.set_params(params_left)
        cost_left = compute_cost(x, y) # L(x; ..., \theta_m - eps, ...)
        
        grads_num[m] = (cost_right - cost_left) / (2 * eps) # 微分の計算
        
        layer.set_params(params) # パラメータをもとに戻す
        shift[m] = 0
    
    grads_all_num.append(grads_num)

#### 2.2. 誤差逆伝播法

In [ ]:
def get_grads(layers):
    grads_all = []
    for layer in layers:
        grads = layer.get_grads()
        grads_all.append(grads)
    
    return grads_all

In [ ]:
# 順伝播
y_hat = f_props(layers, x)

# 逆伝播
delta = y_hat - y
b_props(layers, delta)

# 勾配を取得
grads_all_bprop = get_grads(layers)

#### 2.3. 比較 (勾配チェック)

誤差逆伝播法で計算した勾配と数値微分による勾配の差を、ノルムで正規化したrelative differenceで測ります。[1]

\begin{align*}
    \mathrm{diff} = \frac{||\mathrm{grad}_{\mathrm{bprop}} - \mathrm{grad}_{\mathrm{num}}||_2}{||\mathrm{grad}_{\mathrm{bprop}}||_2 + ||\mathrm{grad}_{\mathrm{bprop}}||_2}
\end{align*}

経験的にはその差がおおよそ$1e-7$以下であれば実装にバグはないと安心して良いでしょう。[1]

[1] Improving Deep Neural Networks: Gradient checking: https://www.coursera.org/lecture/deep-neural-network/gradient-checking-htA0l (2018年10月17日閲覧)

In [ ]:
for i, (grads_bprop, grads_num) in enumerate(zip(grads_all_bprop, grads_all_num)):
    
    diff = np.linalg.norm(grads_bprop - grads_num) / (np.linalg.norm(grads_bprop) + np.linalg.norm(grads_num))
    
    print('Gradients\' difference (layer {}): {}'.format(i + 1, diff))